In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from src.swan import SWAN
from tricorder.procedure_codesets import cabg_names

from src.outcome_metrics import Metric, tidy_flow, tidy_labs, OxygenDelivery, pivot_tidy, melt_tidy

In [2]:
swan = SWAN('/Users/elijahc/data/compass/SWAN/')
eids = swan.procedures.sel(order_name=cabg_names).encounter_id.unique()

In [3]:
from src.outcome_metrics import compute_o2_content

In [4]:
class RespiratoryQuotient(Metric):
    shortname = "RQ"
    classname = 'RespiratoryQuotient'
    REQUIRES = {
        'labs' : [
            'HEMOGLOBIN','HEMOGLOBIN ARTERIAL','HEMOGLOBIN VENOUS',
            'O2SAT ARTERIAL MEASURED','PO2 ARTERIAL',
            'TCO2 VENOUS','TCO2 ARTERIAL','PCO2 ARTERIAL','PH ARTERIAL', 'PH VENOUS','PCO2 VENOUS','BASE EXCESS VENOUS','BASE EXCESS ARTERIAL',
        ],
        # 'flowsheet' : [
        #     'CARDIAC OUTPUT','CCO','CCI',
        # ]
    }
    
    def __init__(self, db, encounter_id=None):
        labs = pd.Series(self.__class__.REQUIRES['labs'])
        self.hgb_names = ['HEMOGLOBIN','HEMOGLOBIN ARTERIAL','HEMOGLOBIN VENOUS']
        self.abg_names = labs[labs.str.contains('ARTERIAL')].tolist()
        self.vbg_names = labs[labs.str.contains('VENOUS')].tolist()
        self.co2_names = ['TCO2 VENOUS','TCO2 ARTERIAL','PCO2 ARTERIAL','PH ARTERIAL', 'PH VENOUS','PCO2 VENOUS','BASE EXCESS VENOUS']
        
        super(RespiratoryQuotient,self).__init__(db, encounter_id)
        
    def compute(self, sample=None, return_components=False):
        if sample is not None and isinstance(sample, int):
            # Draw n values where n=sample
            components = self.db_sample(n=sample).dropna().sort_values(['encounter_id','time'])
            
        elif sample is not None and isinstance(sample, (list, type(np.array([])), type(pd.Series()))):
            # When sample is list assume it's encounter_id's and use that set
            components = self.db_fetch(encounter_id=sample)
            
        else:
            # Otherwise fetch all
            components = self.db_fetch()
        
        grouper = components.groupby('encounter_id')
        pbar = tqdm(grouper)
        
        dfs = []
        
        for _,df in pbar:
            hgb = df[df['name'].isin(self.hgb_names)]
            
            enc_dfs = []
            eid = df.encounter_id.values[0]
            pbar.set_description('{}_a'.format(eid))
            sata = df[df['name'].str.contains('O2SAT ARTERIAL')]
            po2a = df[df['name'].str.contains('PO2 ARTERIAL')]
            aC = compute_o2_content(hgb,sata,po2a)
            aC['name'] = 'CaO2'
            enc_dfs.append(aC)
            
            co = df[df['name'].isin(['CCO','CCI'])].reset_index(drop=True)
            co['day'] = co.time / np.timedelta64(1,'D')
            co = co.drop(columns='time')
            # coi = co[co['name'].isin(['CCI'])].reset_index(drop=True)
            enc_dfs.append(co)

            out = pd.concat(enc_dfs)
            if out.name.drop_duplicates().str.contains('CC').any() and 'CaO2' in out.name.unique():
                out['encounter_id']=eid
                out['hour'] = (out.day*24).round().astype(int)
                dfs.append(out)
            
        output = pd.concat(dfs)[['value','name','encounter_id','hour']].pivot_table(
            index=['encounter_id','hour'], columns='name', values='value'
        )
        output['DO2'] = output['CCO'].interpolate(limit_area='inside')*10*output['CaO2'].interpolate(limit_area='inside')
        output['DO2'] = output['DO2'].interpolate(limit_direction='forward')
        
        output['DO2_I'] = output['CCI'].interpolate(limit_area='inside')*10*output['CaO2'].interpolate(limit_area='inside')
        output['DO2_I'] = output['DO2_I'].interpolate(limit_direction='forward')
        return output
    

SyntaxError: invalid syntax (4099897597.py, line 27)

In [88]:
pc = swan.create_procedure_cohort(procedures=cabg_names)
pc.add_continuous_metric(OxygenConsumption)
pc.add_continuous_metric(CardiacPower)

CohortMetrics
- OxygenConsumption
- CardiacPower


[OxygenConsumption, CardiacPower]

In [ ]:
cont = pc.metrics.OxygenConsumption.compute(sample=60)
# cont['VO2'] = cont['CCI'].interpolate(limit_area='inside') * 10 * (cont['CaO2'].interpolate(limit_area='inside')-cont['CvO2'].interpolate(limit_area='inside'))
cont

In [ ]:
sns.lineplot(x='hour',y='value',hue='name',
             data=melt_metric(cont).query('encounter_id == {}'.format(cont.reset_index().encounter_id.drop_duplicates().values[0])))

In [ ]:
getsatv = lambda d: d[d['name'].isin(['SVO2 (%)','O2SAT VENOUS MEASURED'])]

In [ ]:
vals = pc.metrics.OxygenConsumption.db_sample(2)
sv = vals.groupby('encounter_id').apply(getsatv)
pv = vals.groupby('encounter_id').apply(lambda d: d[d['name'].isin(['PO2 VENOUS'])])
# vals[vals.name.isin(['SCVO2 (%)','SVO2 (%)','O2SAT VENOUS MEASURED'])].pivot_table(index=['encounter_id','time'],values='value', aggfunc='mean', columns='name')

In [ ]:
hgb = vals[vals['name'].isin(pc.metrics.OxygenConsumption.hgb_names)]
hgb

In [ ]:
compute_o2_content(hgb=hgb, sat=sv, po2=pv)

In [ ]:
vals

In [ ]:
pc.metrics.OxygenDelivery.compute(200)

In [ ]:
rq = RespiratoryQuotient(db=swan, encounter_id=eids)

In [ ]:
comp = rq.db_sample(1)
o2_idx = comp.name.str.startswith('O2SAT')
po2_idx = comp.name.str.startswith('PO2')
bg = np.logical_or(o2_idx,po2_idx)
hgb_idx = comp.name.str.startswith('HEMOGLOBIN')
comp[bg].pivot_table(index=['encounter_id','time'],values='value', aggfunc='mean', columns='name')

In [ ]:
swan.labs.search('VENOUS')

In [ ]:
swan.flowsheet.search('SA').values

In [ ]:
swan.flowsheet.sel(display_name=['ETCO2'],encounter_id=eids).flowsheet_value.astype(int).hist()

In [ ]:
rq.vbg_names

In [ ]:
tc = rq.db_sample(n=1)
tc = tc[tc.name.isin(rq.abg_names+rq.vbg_names)]
tc = tc.sort_values(['encounter_id','time'])
tc['hour'] = (tc.time/np.timedelta64(1,'D') * 24).round().astype(int)
tc['rhour'] = tc.groupby('encounter_id').hour.apply(lambda s: s - s.min()).values

freqh = 8
bn = np.arange(-1,tc.rhour.max()+freqh,freqh)
labels = np.arange(freqh,tc
                   .rhour.max()+freqh,freqh)
tc['rhour_b'] = pd.cut(tc.rhour,bins=bn,
                       labels=labels,
                      )
tc
tc_pv = pd.pivot_table(index=['encounter_id','rhour_b'],values='value', aggfunc='mean', columns='name',data=tc)
tc_pv = tc_pv.reset_index()
tc_pv

In [ ]:
swan.labs.unique()

In [ ]:
sns.scatterplot(x='rhour',y='value',hue='name',data=tc)